In [1]:
from coinor.grumpy.polyhedron2D import Polyhedron2D, Figure
import gurobipy as gu
import numpy as np

In [2]:
# utility function to get the model as np arrays
def get_model_arrays(mdl):
    
    # we assume that the model is Ax >= b
    for c in mdl.getConstrs():
        if c.sense == gu.GRB.LESS_EQUAL:
            c.setAttr(gu.GRB.Attr.Sense, gu.GRB.GREATER_EQUAL)
            c.setAttr(gu.GRB.Attr.RHS, -c.rhs)
    
    # get the constraints
    A = mdl.getA().toarray()
    b = np.array([c.rhs for c in mdl.getConstrs()])
    
    # get the finite lower bounds on variables
    Dlb, Dlb_0 = np.eye(mdl.numVars), np.array([v.lb for v in mdl.getVars()])
    Dlb, Dlb_0 = Dlb[Dlb_0 > -np.inf], Dlb_0[Dlb_0 > -np.inf]
    
    # get the finite upper bounds on variables
    Dub, Dub_0 = -np.eye(mdl.numVars), -np.array([v.ub for v in mdl.getVars()])
    Dub, Dub_0 = Dub[Dub_0 > -np.inf], Dub_0[Dub_0 > -np.inf]
    
    # merge the variable bounds into a single matrix with the constraints
    A = np.vstack([A, Dlb, Dub])
    b = np.hstack([b, Dlb_0, Dub_0])
    
    # get the objective function
    c = np.array([v.obj for v in mdl.getVars()])
    
    return A, b, c


# utility function to get the disjunctive constraints as np arrays
def get_disjunctive_constraint_arrays(mdl):
    
    # we assume that the model is Ax >= b
    for c in mdl.getConstrs():
        if c.sense == gu.GRB.LESS_EQUAL:
            c.setAttr(gu.GRB.Attr.Sense, gu.GRB.GREATER_EQUAL)
            c.setAttr(gu.GRB.Attr.RHS, -c.rhs)
    
    # get the disjunctive constraints from branching up
    Dlb, Dlb_0 = np.eye(mdl.numVars), np.array([v.lb for v in mdl.getVars()])
    Dlb, Dlb_0 = Dlb[Dlb_0 > 0], Dlb_0[Dlb_0 > 0]
    
    # get the disjunctive constraints from branching down
    Dub, Dub_0 = -np.eye(mdl.numVars), -np.array([v.ub for v in mdl.getVars()])
    Dub, Dub_0 = Dub[Dub_0 > -np.inf], Dub_0[Dub_0 > -np.inf]
    
    # merge the variable bounds into a single matrix with the constraints
    D = np.vstack([Dlb, Dub])
    D_0 = np.hstack([Dlb_0, Dub_0])
    
    return D, D_0

def get_tableau_primitives(mdl, original_objective=None):
    
    if original_objective is not None:
        assert len(original_objective) == mdl.numVars, \
            "original objective must be the same length as the number of variables"
    
    # create empty containers to hold basis and nonbasis information
    # basis is always m x m since matrix is always m x (n + m)
    A_b = np.zeros((mdl.numConstrs, mdl.numConstrs))
    A_n = np.zeros((mdl.numConstrs, mdl.numVars))
    c_b = np.zeros(mdl.numConstrs)  # always m
    c_n = np.zeros(mdl.numVars)  # always n since we have n + m variables and m are basic
    k_b, k_n = 0, 0  # counter for the number of basic and nonbasic variables
    
    # get the necessary model pieces as arrays
    A = mdl.getA().toarray()
    c = original_objective if original_objective is not None \
        else np.array([v.obj for v in mdl.getVars()])
    
    # populate it with columns from basic decision variables
    for i, v in enumerate(mdl.getVars()):
        if v.vBasis == 0:
            A_b[:, k_b] = A[:, i]
            c_b[k_b] = c[i]
            k_b += 1
        else:
            A_n[:, k_n] = A[:, i]
            c_n[k_n] = c[i]
            k_n += 1
            
    # populate it with columns from basic slack variables
    for i, c in enumerate(mdl.getConstrs()):
        if c.cBasis == 0:
            A_b[i, k_b] = -1
            c_b[k_b] = 0
            k_b += 1
        else:
            A_n[i, k_n] = -1
            c_n[k_n] = 0
            k_n += 1
    
    return A_b, c_b, A_n, c_n

# Define and Solve a Feasible Parent Node's LP

In [3]:
mdl = gu.Model("lp_minimize")

# Create variables
x = mdl.addVar(name="x")
y = mdl.addVar(name="y")

# Set objective function: maximize x + y + 10z
mdl.setObjective(-y, gu.GRB.MINIMIZE)

# Add constraints
constr_0 = mdl.addConstr(-x >= -1, "constr_0")
constr_1 = mdl.addConstr(x - y >= 0, "constr_1")

mdl.update()

# get the model as np arrays
A0 = mdl.getA().toarray()
b0 = np.array([c.rhs for c in mdl.getConstrs()])
c0 = np.array([v.obj for v in mdl.getVars()])

# make sure this works
mdl.optimize()
x.x, y.x

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model fingerprint: 0x60e2fea2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.000000000e+00


(1.0, 1.0)

# Find the Parent Optimal Basis Cone

In [4]:
# get the tableau primitives for the root optimal basis
A_b, c_b, A_n, c_n = get_tableau_primitives(mdl)

In [5]:
# see which decision and slack variables are in the basis (i.e. are not at their bounds - marked by 0 value)
mdl.vBasis, mdl.cBasis

([0, 0], [-1, -1])

In [6]:
# original basis cone
cone_0 = -(np.linalg.inv(A_b) @ A_n)
cone_0

array([[-1., -0.],
       [-1., -1.]])

In [7]:
# x = A_B^-1 b
np.linalg.inv(A_b) @ b0

array([1., 1.])

In [10]:
c_b, np.linalg.inv(A_b) @ A_n, c_n

(array([ 0., -1.]),
 array([[1., 0.],
        [1., 1.]]),
 array([0., 0.]))

In [8]:
# -s_N = c_B A_B^-1 A_N - c_N
np.dot(c_b, np.linalg.inv(A_b) @ A_n) - c_n

array([-1., -1.])

# Calculate Reduced Costs Relative to Original Objective

In [13]:
# here are our reduced costs - makes sense as c2 and c3 are tight
# negative reduced cost =>'s tightening the constraint would improve the objective
# we want to get rid of the constraint where tightening would improve the objective
# therefore, we want to get rid of c2
# y = A_B^-T c_B
row_duals = np.linalg.inv(A_b.T) @ c_b
row_duals

array([-3.39827881e-18,  0.00000000e+00,  0.00000000e+00, -1.35272847e-16,
        0.00000000e+00, -3.55840713e-20, -1.38777878e-16,  0.00000000e+00,
       -9.79166667e-01, -1.47916667e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  4.03846154e-02])

In [14]:
# these are the same as nonzero row duals if all nonbasic variables are slacks
# s_N = c_N - c_B A_B^-1 A_N
col_duals = -(A_n.T @ row_duals - c_n)  # use the column duals when decision variable is nonbasic?
col_duals

array([-0.97916667, -1.47916667,  0.04038462])

In [15]:
# want to make sure that the reduced costs are zero for all basic slack variables
assert np.all(np.abs(row_duals[np.array(mdl.cbasis) == 0]) < 1e-7), \
    "all constraints that are slack should have 0 reduced cost"

# get most negative reduced cost for nonbasic decision and slack variables
decision_idx = np.argmin(col_duals[np.array(mdl.vbasis) == -1]) if np.any(np.array(mdl.vbasis) == -1) else None
decision_rc = col_duals[np.array(mdl.vbasis) == -1][decision_idx] if decision_idx is not None else np.inf
slack_idx = np.argmin(row_duals[np.array(mdl.cbasis) == -1]) if np.any(np.array(mdl.cbasis) == -1) else None
slack_rc = row_duals[np.array(mdl.cbasis) == -1][slack_idx] if slack_idx is not None else np.inf

# print the most negative reduced cost and the variable that corresponds to it over all nonbasic variables
nbd_idxs = [i for i, status in enumerate(mdl.vbasis) if status == -1]  # nonbasic decision variable indices
nbs_idxs = [i for i, status in enumerate(mdl.cbasis) if status == -1]  # nonbasic slack variable indices
replace_slack = decision_rc >= slack_rc
if replace_slack:
    print(f"replace nonbasic slack var {nbs_idxs[slack_idx]}")
else:
    print(f"replace nonbasic decision var {nbd_idxs[decision_idx]}")

replace nonbasic slack var 9


# Pivot Out Variable with Most Reduced Cost for Slack Variable of Disjunctive Constraint

In [16]:
# copy the original model so we can append to it
A1, b1, c1 = A0, b0, c0

# add the disjunctive constraint y >= 1
A1 = np.vstack([A1, np.array([0, 1, 0])])  
b1 = np.hstack([b1, 1])
c1 = np.hstack([c1, 0])

In [17]:
# create empty containers for tableau
# basis is always m x m since matrix is always m x (n + m)
A_b = np.zeros((mdl.numConstrs + 1, mdl.numConstrs + 1))
A_n = np.zeros((mdl.numConstrs + 1, mdl.numVars))
c_b = np.zeros(mdl.numConstrs + 1)
c_n = np.zeros(mdl.numVars)
k_b, k_n = 0, 0  # counter for the number of basic and nonbasic variables
v_basis, c_basis = np.zeros(mdl.numVars), np.zeros(mdl.numConstrs + 1)  # basis vector

# populate it with columns from basic decision variables
for i, v in enumerate(mdl.getVars()):
    # either what was already basic or the new decision variable being pivoted in
    if v.vBasis == 0 or (not replace_slack and i == nbd_idxs[decision_idx]):
        A_b[:, k_b] = A1[:, i]
        c_b[k_b] = c1[i]
        k_b += 1
    else:
        A_n[:, k_n] = A1[:, i]
        c_n[k_n] = c1[i]
        k_n += 1
        v_basis[i] = -1
        
# populate it with columns from basic slack variables
for i, c in enumerate(mdl.getConstrs()):
    # either what was already basic or the new slack variable being pivoted in
    if c.cBasis == 0 or (replace_slack and i == nbs_idxs[slack_idx]):
        A_b[i, k_b] = -1
        c_b[k_b] = 0
        k_b += 1
    else:
        A_n[i, k_n] = -1
        c_n[k_n] = 0
        k_n += 1
        c_basis[i] = -1
    
# now forcibly add the disjunctive constraint's slack variable to the nonbasis
assert k_b == mdl.numConstrs + 1 and k_n == mdl.numVars - 1, \
    "the final basis status to fill should be a nonbasic one for the last slack variable"
A_n[mdl.numConstrs, k_n] = -1
k_n += 1
c_basis[mdl.numConstrs] = -1

# Calculate Resulting Infeasible Basis Cone

In [18]:
cone_1 = -(np.linalg.inv(A_b) @ A_n)[:mdl.numVars, :mdl.numVars]
cone_1

array([[ -0.5       ,  -0.02307692, -12.        ],
       [ -0.        ,  -0.        ,   1.        ],
       [ -0.        ,  -0.01538462,  -6.        ]])

# Calculate Multipliers of Parent Optimal Basis Cone Generating Each Ray in Infeasible Basis Cone

Let $B \in \mathbb{Z}^{m}$ and $N \in \mathbb{Z}^{n}$ represent the indices of basic and nonbasic variables at a basic solution.
Let $R \in \mathbb{R}^{n \times n}$ be such that for $i \in n$ we have that $R_{i,j} = -(A_B^{-1}A_N)_{i,j}$. I.e. $R_{*,j}$ represents the $j^{\text{th}}$
ray in the basis cone formed by the intersection of constraints active for basis $B$ (which are those whose slack variables' indices are in $N$).

Consider $B^1$ and $N^1$ as well as $B^2$ and $N^2$, which are, respectively, the basis and nonbasis for two separate basic solutions, and
$R^1$ and $R^2$ as matrices representing their corresponding basis cones. For $j \in n$, the $j^{\text{th}}$ ray of basis cone 2, $R^2_{*,j}$,
belongs to basis cone 1 if and only if there exists a vector $\gamma^j \in \mathbb{R}^{n}_{\geq 0}$ such that $R^1\gamma^j = R^2_{*,j}$, i.e. is a convex
combination of the rays defining $R^1$ (this follows from properties of convexity and linear independence of constraints constituting a basis).   

Again by linear independence of our bases, $\gamma^j = (R^1)^{-1} R^2_{*,j}$ is the unique set of multipliers of the rays of basis cone 1 that generate
the $j^{\text{th}}$ ray of basis cone 2. If any of the components of $\gamma^j$ are negative, then the $j^{\text{th}}$ ray of basis cone 2 is not
contained in basis cone 1, and by extension, basis cone 2 is not a subset of basis cone 1.

For ease of implementation, we can compute the multipliers of the rays of basis cone 1 that generate the rays of basis cone 2 by solving the system
$\Gamma = (R^1)^{-1} R^2$, where $\Gamma_{*,j} = \gamma^j$, and checking if any of the components of $\Gamma$ are negative, which we do below.

In [19]:
# get the multipliers of the rays of the parent basis cone that generate the rays of the infeasible child's basis cone
a = np.linalg.inv(cone_0) @ cone_1
a

array([[ 1.00000000e+00,  9.23076923e-02,  4.20000000e+01],
       [-1.00000000e+00,  2.13504428e-19, -6.00000000e+00],
       [ 1.00000000e+00,  9.23076923e-02,  3.60000000e+01]])

In [20]:
if (a < -1e-6).any():
    print("negative multiple/s of parent basis cone ray/s required to generate infeasible basis cone")
    print("infeasible basis cone is not a subset of parent basis cone")

negative multiple/s of parent basis cone ray/s required to generate infeasible basis cone
infeasible basis cone is not a subset of parent basis cone


# Check Primal and Dual Feasibility Statuses

In [21]:
# check primal feasibility
np.linalg.inv(A_b) @ b1

array([-3.7000000e+00,  1.0000000e+00, -1.8000000e+00,  1.6800000e+01,
        4.7000000e+00,  3.0000000e+00, -1.3800000e+01, -3.7000000e+00,
       -8.8817842e-16,  1.3800000e+01,  3.7000000e+00, -1.6800000e+01,
       -4.7000000e+00, -3.0000000e+00,  1.0000000e+00])

In [22]:
# check dual feasibility
np.dot(c_b, np.linalg.inv(A_b) @ A_n) - c_n

array([ -0.5       ,  -0.17692308, -71.        ])